In [1]:
import pyspark 
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [2]:
from pyspark.sql.functions import *
from pyspark.sql import *
from pyspark.sql.types import *

## s3 bucket name --> yelp_data_parquet , folder--> parquet [store file from kaggle]

## s3 bucket name --> eda_parquet , folder--> parquet  [storing file after doing EDA]


# Reading Business file

In [ ]:
business = spark.read.format("parquet") \
                         .option("header", "true") \
                         .option("inferSchema", "true") \
                         .option("quote", "\"")\
                         .option("escape", "\"")\
                         .load("s3://yelp_data_parquet/parquet/business.parquet")

In [ ]:
business.cache()

In [ ]:
business1 = business.na.fill(value='Edinburgh',subset=["city"])

In [ ]:
business2 = business1.na.fill(value='',subset=["neighborhood"])

In [ ]:
business3 = business2.na.fill(value='NV',subset=["State"])

In [ ]:
business4 = business3.na.fill(value='',subset=["postal_code"])

In [ ]:
business5 = business4.na.fill(value=43.651070 , subset=["Latitude"])

In [ ]:
business6 = business5.na.fill(value=-79.347015 , subset=["longitude"])

## saving file

# Reading Review

In [ ]:
review_schema = StructType([
    StructField("review_id", StringType()),
    StructField("user_id", StringType()),
    StructField("business_id", StringType()),
    StructField("stars", FloatType()),
    StructField("date", StringType()),
    StructField("text", StringType()),
    StructField("useful", IntegerType()),
    StructField("funny", IntegerType()),
    StructField("cool", IntegerType())
])

In [ ]:
review= spark.read.format("parquet") \
                         .option("header", "true") \
                         .schema(review_schema) \
                         .option("quote", "\"")\
                         .option("escape", "\"")\
                         .option("dateFormat", "yyyy-MM-dd HH:mm:ss")\
                         .option("multiLine",'true')\
                         .load("s3://yelp_data_parquet/parquet/review.parquet")

## saving file

# Reading Checkin

In [ ]:
checkin = spark.read.format("parquet") \
                         .option("header", "true") \
                         .option("inferSchema","true")\
                         .load("s3://yelp_data_parquet/parquet/checkin.parquet")

## saving file

## Reading Tip

In [ ]:
tip_schema = StructType([
    StructField("text", StringType()),
    StructField("date", StringType()),
    StructField("likes", IntegerType()),
    StructField("business_id", StringType()),
    StructField("user_id", StringType()),
])

In [ ]:
tip = spark.read.format("parquet") \
                         .option("header", "true") \
                         .schema(tip_schema)\
                         .option("quote", "\"")\
                         .option("escape", "\"")\
                         .option("multiLine",'true')\
                         .load("s3://yelp_data_parquet/parquet/tip.parquet")

In [ ]:
tip1 = tip.na.fill(value='',subset=["text"])

## saving file

## Reading Hours

In [ ]:
hours = spark.read.parquet("s3://yelp_data_parquet/parquet/hours.parquet",header=True)

In [ ]:
from pyspark.sql.functions import regexp_replace
hours1 = hours.withColumn('monday', regexp_replace('monday', 'None', ''))
hours2 = hours1.withColumn('tuesday', regexp_replace('tuesday', 'None', ''))
hours3 = hours2.withColumn('wednesday', regexp_replace('wednesday', 'None', ''))
hours4 = hours3.withColumn('thursday', regexp_replace('thursday', 'None', ''))
hours5 = hours4.withColumn('friday', regexp_replace('friday', 'None', ''))
hours6 = hours5.withColumn('saturday', regexp_replace('saturday', 'None', ''))
hours7 = hours6.withColumn('sunday', regexp_replace('sunday', 'None', ''))

## saving file

# Reading user

In [ ]:
user = spark.read.format("parquet") \
                         .option("header", "true") \
                         .option("inferSchema", "true") \
                         .load("s3://yelp_data_parquet/parquet/user.parquet")

In [ ]:
user.cache()

In [ ]:
user2 = user1.na.fill(value='Unknown',subset=["name"])


In [ ]:
user2 = user1.na.fill(value='Unknown',subset=["name"])

In [ ]:
user3 = user2.withColumn('name', regexp_replace('name', 'None', 'Unknown'))

In [ ]:
user4 = user3.withColumn('elite', regexp_replace('elite', 'None', ''))

In [ ]:
user5 = user4.withColumn('friends', regexp_replace('friends', 'None', ''))

In [ ]:
user6=user5.drop('fans','compliment_hot',
 'compliment_more',
 'compliment_profile',
 'compliment_cute',
 'compliment_list',
 'compliment_note',
 'compliment_plain',
 'compliment_cool',
 'compliment_funny',
 'compliment_writer',
 'compliment_photos')


In [ ]:
from pyspark.sql.functions import split

In [ ]:
user7 = user6.withColumn("Yelping_date", split(col("Yelping_since"), " ").getItem(0)).withColumn("col2", split(col("Yelping_since"), " ").getItem(1))

In [ ]:
user8=user7.drop('Yelping_since','col2')

In [ ]:
user9= user8.withColumnRenamed('Yelping_date', 'Yelping_since') # renaming  column yelping date as yelping_since

In [ ]:
user10 = user9.select('user_id','name','review_count','Yelping_since','friends','useful','funny','cool','elite','average_stars')

## saving file